In [1]:
import pandas as pd

In [21]:
fga = pd.read_csv('NBA_22_23_FGA.csv', sep=';')
fga.head()

,PLAYER,PLAY TYPE,MADE,SHOT TYPE,BOXSCORE,VTM,HTM,Game Date,PERIOD,TIME REMAINING,SHOT DISTANCE (FT),TEAM
0,Marcus Smart,Driving Floating Bank Jump Shot,✔ Made Shot,2PT Field Goal,PHI @ BOS,PHI,BOS,"Tuesday, October 18",1,11:15,13,Boston Celtics
1,Jayson Tatum,Jump Shot,✔ Made Shot,3PT Field Goal,PHI @ BOS,PHI,BOS,"Tuesday, October 18",1,10:46,23,Boston Celtics
2,Derrick White,Running Layup Shot,✖ Missed Shot,2PT Field Goal,PHI @ BOS,PHI,BOS,"Tuesday, October 18",1,10:04,1,Boston Celtics
3,Al Horford,Cutting Layup Shot,✖ Missed Shot,2PT Field Goal,PHI @ BOS,PHI,BOS,"Tuesday, October 18",1,09:53,2,Boston Celtics
4,Jayson Tatum,Running Layup Shot,✔ Made Shot,2PT Field Goal,PHI @ BOS,PHI,BOS,"Tuesday, October 18",1,09:42,0,Boston Celtics


In [41]:
## Tabela com o nome e o código de todos os time da NBA
cod_teams = pd.read_csv('cod_teams.csv', sep=';')
cod_teams.head()

,name,cod
0,Atlanta Hawks,ATL
1,Boston Celtics,BOS
2,Brooklyn Nets,BKN
3,Charlotte Hornets,CHA
4,Chicago Bulls,CHI


In [22]:
## Alguns tratamentos que achamos prudentes:
## 1) Substituir "✔ Made Shot" por 1 e "✘ Missed Shot" por 0

fga['MADE'] = fga['MADE'].replace('✔ Made Shot', 1)
fga['MADE'] = fga['MADE'].replace('✖ Missed Shot', 0)


## 2) Substituir "2PT Field Goal" por 2 e "3PT Field Goal" por 3

fga['SHOT TYPE'] = fga['SHOT TYPE'].replace('2PT Field Goal', 2)
fga['SHOT TYPE'] = fga['SHOT TYPE'].replace('3PT Field Goal', 3)


## 3) Criar uma coluna para identificar se o time que arremessou é mandante ou visitante da partida

is_home = []
for i in range(len(fga)):
    if (cod_teams[cod_teams['name'] == fga['TEAM'][i]]["cod"].values[0]) == fga['HTM'][i]:
        is_home.append(1)
    else:
        is_home.append(0) 
fga['IS_HOME'] = is_home


## 4) Transformar a coluna "TIME REMAINING" em segundos

time_remaining = []
for i in range(len(fga)):
    time = fga['TIME REMAINING'][i].split(':')
    time_remaining.append(int(time[0])*60 + int(time[1]))

fga['TIME REMAINING'] = time_remaining
fga.head()



In [69]:
fga.rename(columns={'Game\xa0Date': 'Game Date'}, inplace=True)
fga

,PLAYER,PLAY TYPE,MADE,SHOT TYPE,BOXSCORE,VTM,HTM,Game Date,PERIOD,TIME REMAINING,SHOT DISTANCE (FT),TEAM,IS_HOME
0,Marcus Smart,Driving Floating Bank Jump Shot,1,2,PHI @ BOS,PHI,BOS,"Tuesday, October 18",1,675,13,Boston Celtics,1
1,Jayson Tatum,Jump Shot,1,3,PHI @ BOS,PHI,BOS,"Tuesday, October 18",1,646,23,Boston Celtics,1
2,Derrick White,Running Layup Shot,0,2,PHI @ BOS,PHI,BOS,"Tuesday, October 18",1,604,1,Boston Celtics,1
3,Al Horford,Cutting Layup Shot,0,2,PHI @ BOS,PHI,BOS,"Tuesday, October 18",1,593,2,Boston Celtics,1
4,Jayson Tatum,Running Layup Shot,1,2,PHI @ BOS,PHI,BOS,"Tuesday, October 18",1,582,0,Boston Celtics,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209621,Herbert Jones,Tip Layup Shot,0,2,NOP @ MIN,NOP,MIN,"Sunday, April 9",4,101,0,New Orleans Pelicans,0
209622,CJ McCollum,Driving Dunk Shot,0,2,NOP @ MIN,NOP,MIN,"Sunday, April 9",4,68,4,New Orleans Pelicans,0
209623,Herbert Jones,Driving Dunk Shot,1,2,NOP @ MIN,NOP,MIN,"Sunday, April 9",4,17,1,New Orleans Pelicans,0
209624,CJ McCollum,Layup Shot,0,2,NOP @ MIN,NOP,MIN,"Sunday, April 9",4,11,4,New Orleans Pelicans,0


In [43]:
## Quais são os times que mais arriscam arremessos?

fga.value_counts('TEAM').head(5)

TEAM
Oklahoma City Thunder    7590
Atlanta Hawks            7574
Memphis Grizzlies        7551
Toronto Raptors          7489
Charlotte Hornets        7413
dtype: int64

In [44]:
## E os que mais possuem arremessos convertidos?

fga[fga["MADE"] == 1].value_counts('TEAM').head(5)

TEAM
Atlanta Hawks            3658
Memphis Grizzlies        3585
Denver Nuggets           3574
Sacramento Kings         3573
Golden State Warriors    3538
dtype: int64

In [45]:
## Quais são os times com maior taxa de acerto nos arremessos?

(fga[fga["MADE"] == 1].value_counts('TEAM') / fga.value_counts('TEAM')).sort_values(ascending=False).head(5).map('{:.2%}'.format)

TEAM
Denver Nuggets            50.42%
Sacramento Kings          49.41%
Minnesota Timberwolves    49.04%
Chicago Bulls             49.02%
Cleveland Cavaliers       48.80%
dtype: object